# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'baseball', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [8]:
# review a document in the establishments collection
print(db.establishments.find_one({}))

{'_id': ObjectId('67a40905c129f9cfdf7eed2e'), 'FHRSID': 254719, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000069980', 'BusinessName': 'Refreshment Kiosk', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'The Bay', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DY', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-03-24T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'}, 'RightToReply': '', 'Distance': 4587.347174863443, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'total

In [9]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [11]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('67a40bbff651cb02255b33b3'), acknowledged=True)

In [13]:
# Check that the new restaurant was inserted
query1 = {"BusinessName":"Penang Flavours"}
results = establishments.find(query1)

for result in results:
    print(result)

{'_id': ObjectId('67a40bbff651cb02255b33b3'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [15]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query2 = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {'BusinessName':1, 'BusinessTypeID':1, 'BusinessType':1}
          
# Capture the results to a variable
results = establishments.find(query2, fields)

# Pretty print the first two results
df = pd.json_normalize(results)
df.head()

,_id,BusinessName,BusinessType,BusinessTypeID
0,67a40905c129f9cfdf7eed2e,Refreshment Kiosk,Restaurant/Cafe/Canteen,1
1,67a40905c129f9cfdf7eed31,The Tea Room,Restaurant/Cafe/Canteen,1
2,67a40905c129f9cfdf7eed33,Mrs Knotts Tea Room,Restaurant/Cafe/Canteen,1
3,67a40905c129f9cfdf7eed34,Boodles,Restaurant/Cafe/Canteen,1
4,67a40905c129f9cfdf7eed35,FirstLight Bar & Café,Restaurant/Cafe/Canteen,1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [16]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"}, {'$set':{'BusinessTypeID':1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [18]:
# Confirm that the new restaurant was updated
query1 = {"BusinessName":"Penang Flavours"}
results = establishments.find(query1)

for result in results:
    print(result)

{'_id': ObjectId('67a40bbff651cb02255b33b3'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [19]:
# Find how many documents have LocalAuthorityName as "Dover"
query3 = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query3)

994

In [20]:
# Delete all documents where LocalAuthorityName is "Dover"
query3 = {"LocalAuthorityName":"Dover"}
establishments.delete_many(query3)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [21]:
# Check if any remaining documents include Dover
query3 = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query3)

0

In [22]:
# Check that other documents remain with 'find_one'
print(db.establishments.find_one({}))

{'_id': ObjectId('67a40905c129f9cfdf7ef015'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [25]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, 
                           [ 
                               {'$set':{ 
                                   "geocode.longitude" : {'$toDouble': "$geocode.longitude"}, 
                                   "geocode.latitude" : {'$toDouble': "$geocode.latitude"}
                                         }} 
                           ])

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [26]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [27]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, 
                           [ 
                               {'$set':{ 
                                   "RatingValue" : {'$toInt': "$RatingValue"}
                               }} 
                           ])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [28]:
# Check that the coordinates and rating value are now numbers
print(db.establishments.find_one({}))

{'_id': ObjectId('67a40905c129f9cfdf7ef015'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'retur